## Сшивание изображений ##

Так как задача в целом весьма стандартная, собирать будем из стандартных же кусков. За основу взят алгоритм статьи: [Brown, Lowe. Automatic Panoramic Image Stitching using Invariant Features](http://matthewalunbrown.com/papers/ijcv2007.pdf) По сути, большинство найденных мной по этой теме текстов все равно ссылаются на эту работу.

### Анамнез ###
1. все изображения каждого набора лежат на одной плоскости (так как нет видимых перспективных искажений узнаваемых объектов aka крупных пятен), а значит совмещаются аффинным преобразованием.
2. В первом наборе данных экспозиция всех изображений одинаковая, во втором потребуется коррекция (гамма или гейн, будем смотреть в процессе; логарифмическая мало вероятна, но будем держать в голове)
3. в условиях обещано последовательное наложение. Этим можно существенно воспользоваться, совмещая ключевые точки только соседних изображений. Такой подход в боевой ситуации неизбежно приведет к инерционной ошибке, но на синтетических данных может сработать. Если не сработает, можно попробовать easy fix: совмещать два последние изображения как начальное приближение, находить еще одно-два изображения с максимальным геометрическим пересечением с новым кусочком (назовем из вспомогательными) и повторить matching ключевых точек предпоследнего+вспомогательного изображения с последним. Такая поправка должна хорошо сработать, если изображение нарезано "змейкой" и хуже, если спиралью. В боевом применении стоит вместо этого использовать bundle adjustment из статьи выше. На синтетических данных есть надежда, что даже матчинг с последним изображением будет работать хорошо. Опять же, вспоминаем пункт 1: линейные преобразования должны давать более-менее линейную же ошибку округлений. 

### Основные шаги алгоритма: ###
1. поиск ключевых точек посредством DoG
2. построение SIFT-дескрипторов. SURF должен быть быстрее, но в python-opencv версии 4.5.5, которую я использовал, его еще нет (только в opencv-contrib). Скорость работы SIFT оказалась приемлемой и я не стал чинить то, что работает.
3. Совмещение инвариантных представлений осуществляется при помощи FLANN. 
4. Далее, найденные совмещения фильтруются, чтобы исключить далекие пары (FLANN находит ближайшего соседа, даже если он не близок...) и точки вне пересечения (RANSAC) и по ним строится приближенное проективное преобразование (на самом деле аффинное).
5. В работе [Brown-Lowe] коррекцию экспозиции (gain) предлагается производить, минимизируя квадратичный функционал на перекрывающихся кусках. Мы упростим эту схему, и будем приводить гейн каждого последующего изобржения к предыдущему посредством вычисления среднего отношения пикселей на пересекающихся участках. Гамма-коррекция для имеющихся наборов данных не пондобилась. Если первое изображение в серии было недосвечено, такая операция недосвечивает все последующие, поэтому добавлена возможность глобальной коррекции: если ее включить, то все коэффициенты коррекции гейна сохранаются и в финале усредняются, после чего к итоговому изобажению применяется коррекция на обратную величину. Такой подход имеет проблемы (см. ниже)
6. К новому куску применяется гомография и он вклеивается к предыдущим

Пункты 1-4 реализуются стандартно при помощи opencv 4.5.5, большинство параметров взято либо по умолчанию, либо со stackoverflow. Пункт 6 требует задуматься об ориентации накопленного и добавляемого изображений на подложке. Я вижу здесь несколько подходов:
1. вращать (здесь и далее под вращением подразумевается произвольная гомография) накопленное изображение вокруг нового, тогда новое можно дописывать в левый верхний угол, а размер подложки можно вычислить, посчитав координаты углов bounding box накопленного изображения. Здесь есть два существенных минуса: а) повторяющиеся вращения одного и того же изображения приведут к накоплению интерполяционных ошибок. б) вращение большого накопленного изображения ресурсозатратно.
2. вращать новый кусочек вокруг накопленного изображения. Здесь основная проблема в том, что в координатах накопленного изображения образ нового куска может быть далеко и совмещать два больших изображения, что не только небыстро, но и требует, условно, удвоения используемой памяти. Я решаю эту проблему так: из гомографии выделяется трансляция, после чего заменяется на малую трансляцию, достаточную для расширения подложки, гарантирующее, что в нее поместится результат любого вращения, затем применяется получившаяся новая гомография и результат вклеивается alpha-blending'ом с учетом необходимого смещения.

### Допущения и попущения ###

В целях экономии времени я отказываюсь от вычисления от пересчета необходимой подложки и задаю отступы для первого изображения в серии вручную. В рамках тестового задания мне не кажется это большим грехом, т.к. поправить это очень легко: достаточно перед вклейкой очередного изображения вычислить координаты образа его центра и, если они окажутся отрицательными, сместить накопленное изображение на подложке вправо и вниз. В конце нужно найти наименьший прямоугольник (возможно, развернутый), содержащий результат (маска результата содержится в alpha-канале) и развернуть его так, чтобы одна из сторон стала горизонтальной, после чего обрезать все за пределами прямоугольника. Также, я не стал проводить коррекцию инерционной ошибки (предложение, как это сделать, есть выше, но результат кажется мне достаточно хорошим и без этого).

Аналогично, вместо минимальной gain-коррекции по всем изображениям, каждое последующее
приводится к gain-уровню накопленного.

По той же причине я не занимаюсь аккуратным логированием и обработкой исключений.

### Очевидные проблемы ###

Имеющийся подход хорошо работает только на синтетических данных, на которых гомография "приближенно" считается очень точно. На реальных данных нужно обязательно пересчитывать ее относительно как минимум двух разных изображений.

Кроме того, есть глупая проблема с белым цветом. Предположим, что изображений всего два: A и B и на их границе есть белая область. Пусть изображение B пересвечено. При этом в каждой белой точке происходит переполнение 8-битного числа и оно срезается до 255. Теперь, когда мы вичислим среднее попиксельное отношение A/B на их пересечении мы получим значение gain_corr что-то строго меньше 1. Умножая каждую точку изображения B на gain_corr получаем не белый цвет на пересечении. 

Это можно как минимум частично пофиксить при помощи multiband blending из статьи, но реализовать его в рамках моей схемы сложно: там коэффициент яркости считается как Гауссово ядро с центром в каждом отельном кусочке изображения (в моем алгоритме склейка производится локально), а кроме того используются разные коэффициенты в разных частях частотного диапазона. 

Интерполяция при поворотах оставляет ошибку на границе изображения, что выражается в наличии темных полос шириной в 2 пикселя. Бороться с этим легко: изображение нужно расширить на 2 пикселя и зеркально повторить в них имеющиеся пиксели, затем повернуть и обрезать несколько крайних пикселей (повернутой рамки). Этого должно хватить, но, опять же, не хватает времени на реализацию.

### Артефакты и метрики ###
На синтетических данных алгоритм достаточно хорошо совмещает геометрию, а alpha-blending дает худо-бедно удовлетворительный результат по цветам с учетом указанной выше проблемы. В сочетании с тем, то моя реализация завершается за 7 минут, а стандартный ститчер в opencv не завершается вообще, я считаю, что для тестового задания этого вполне хватает. 

В качестве оценки сшивания можно использовать MSE разностей пикселей в каждой области пересечения после коррекции гейна. Из-за указанной проблемы с белым цветом следует ожидать не слишком хорошего результата (все белые пиксели на пересечении изображений с разным гейном сделают свой вклад в ошибку). С этим можно побороться, усреднив по всем пересечениям дисперсию разностей пикселей. В этом случае влияние постоянной составляющей будет снижено.

Все эти метрики оценивают только качество совмещения, чтобы оценить блендинг нужно как-то смотреть на производные (условно, есть ли скачки в месте склеек), но эту тему нужно изучать отдельно.

### Реализация в opencv ###
В opencv реализован end-to-end алгоритм такого сорта -- cv2.Stitcher, но во-первых это было бы совсем уж неспортивно, а во-вторых, он не справляется: на малом количестве изображений он выдает кривулю, так как то ли счиает изображения панорамой с камеры и пытется их выпрямить, игнорируя наклоны, то ли пытается компенсировать бочкообразные искажения. А начиная с некоторого количество изоражений выжирает прорву памяти и люто тормозит. Я оставлял его на ночь с полной пачкой изображений из первого набора и он в какой-то момент просто упал. Так что точно придется собирать руками.


In [1]:
import data4
from timeit import default_timer as timer
import stitcher
import cv2
import os, os.path

In [2]:
data_dir = "../04_image_matching/imgage-matching-samples-v2"

#Uncomment unext 2 lines to use the first set of images (66 imgs)
img_csv = "O00401574_010_HE_clip2p1.tif-crops-c0-idx.txt"
target_offsets = (300, 7000, 1000, 9500)

#Uncomment unext 2 lines to use the second set of images (50 imgs, uneven gain)
# img_csv = "O00401574_010_HE_clip3.tif-crops-c1-idx.txt"
# target_offsets = (0, 6000, 630, 8000)

images = data4.StitchingDataset(data_dir, img_csv)

out_dir = "out4"
if not os.path.exists(out_dir): os.mkdir(out_dir)

In [3]:
'''
    Warning: when num_images is set to high values this cell takes A LOT of time to execute.
    On my machine setting num_images = len(images) (66 for the first dataset),
    this code couldn't complete in 1 whole hour. The time complexity is nonlinear
    10 images are precessed in 25 seconds
    20 images are precessed in 245 seconds
    Thus, the time complexity is more than cubic in terms 
    number of input images (of the same size)!

    Here and after, my hardware setup is:
    i5-7300 4@2.5Ghz
    8GB RAM
    HDD
    GTX1050 2Gb (not used by cv2 though...)
'''
#uses opencv 4.5.5, previous versions had differen signatures lie createStitcher or Stitcher_create
cv2stitcher = cv2.Stitcher.create() 
start = timer()
num_images = 5#len(images)
rgb_images = [img[:,:,:3] for img in images[0:num_images]]
status, cv2result = cv2stitcher.stitch(rgb_images)
end = timer()
print(f"Stitched in: {end-start} seconds")
if not status:
    cv2.imwrite(f"{out_dir}/deflt_stitcher_result.png", cv2result)
else:
    print(f"Can't stich. Status code: {status}")

Stitched in: 12.5892816 seconds


In [ ]:
#Сustom stitcher
SIFT_limit = 10000 #limits the number of generated features to n best ones
mystitcher = stitcher.Stitcher(SIFT_limit)

start = timer()
num_images = len(images)


myresult = mystitcher.stitch(images[:num_images], target_offsets, write_intermediate=False,
    output_filename=f"{out_dir}/stitched", 
    gain_correction = stitcher.GainCorrection.NONE, compute_mse=True)
end = timer()
print(f"Stitched in: {end-start} seconds")
if (len(myresult) == 2):
    print(f"MSE: {myresult[1]}")

Stitching:   0%|          | 0/66 [00:00<?, ?it/s]

Экспериментально подтверждается, что проблема белого цвета сбивает mse как метрику. На втором наборе изображений (50 картинок) у меня получились следующие результаты. Время указано с учетом отключенного сохранения промежуточных картинок и вычисления mse (кроме первой строки), 
`target_offsets = (0, 6000, 600, 8000)`. </br>

    gain correction     | mse computed | time min:sec  | mse       | sqrt(mse)
    ==========================================================================
    NONE                | yes          | 3:24          | 274.58    | 16.57
    NONE                | no           | 2:58          | 274.58    | 16.57
    LOCAL               | no           | 4:20          | 169.98    | 13.04
    LOCAL_AND_GLOBAL    | no           | 4:37          | same      | same

### Результаты для второго набора (разрешение уменьшено) ###
**Без коррекции гейна**
![](stitched_NONE.jpg)

**Локальная коррекция**
![](stitched_LOCAL.jpg)

**Локальная и глобальная коррекция**
![](stitched_LOCAL_AND_GLOBAL.jpg)

**Картинка из первого набора** (коррекция гейна отключена)
![](stitched_1.jpg)

### Занятная белиберда ###
Была у меня глупая идея (одна из многих) побороться с проблемой белого цвета аппроксимируя
гейн гамма-коррекцией. Идея была в том, что если отклонение малое, то корень будет близок к прямой, при этом гамма-коррекция всегда переводит белый в белый (1 в любой степени 1). Вот, чо из этого вышло:
![](stitched_wrong_gamma.jpg)

Действительно, при малых отклонениях это более-менее работает, но как только отклонения увеличиваются, получается совсем плохо (цвета съехали из-за поканального вычисления параметров; впоследтви я отказался от поканальной коррекции и для коррекции гейна, но раз уж картнка представлена как плохая, пусть будет совсем плохая).